## Etude des sujets des amendements PLFSS

## Préparation des données

In [ ]:
#!python -m pip install --upgrade pip

In [ ]:
#!pip install -q --upgrade pandas  spacy  more_itertools

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
!python -m spacy download fr_core_news_sm > /dev/null

In [ ]:
# Importing modules
import pandas as pd
#amdt = pd.read_csv('textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdt = pd.read_csv('https://github.com/leximpact/donnees-extraites-assemblee/raw/main/textes_amendements_nouveaux_articles_plfss_2020-2021.csv')
amdt.head(5)

,texteLegislatifUid,uid,avantAApres,dispositif,exposeSommaire
0,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000001,A,"I. – Après l’alinéa 13, insérer l’alinéa suiv...",La mise en place d’un accord d’intéressement d...
1,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000002,Apres,À la première phrase du premier alinéa de l’ar...,L’article L 531‑2 du Code de la Sécurité Socia...
2,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000005,A,Compléter l’alinéa 17 par la phrase suivante :...,S’il est louable d’expérimenter des dispositif...
3,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000008,A,"Après l’alinéa 8, insérer les cinq alinéas sui...",Cet amendement permet d’amplifier la portée de...
4,PRJLANR5L15B2296,AMANR5L15PO420120B2296P0D1N000010,Apres,Le premier alinéa de l’article L. 521‑1 du cod...,"Pendant plus de cinquante ans, notre politique..."


In [ ]:
len(amdt) #Nombre total d'amendements

4797

In [ ]:
#On regroupe dans un même texte chaque dispositif et son exposé sommaire
amdt['texte'] = amdt['dispositif'] + amdt['exposeSommaire'] 

## Nettoyage

In [ ]:
#pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/ben/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to /home/ben/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Tokenisation := découpage du texte en listes de mots
from nltk.tokenize import word_tokenize
tokenized = [ word_tokenize(text) for text in amdt['texte'] ] #return_token

# Normalization

### Retrait des mots de liaison (_stopword_) et de la ponctuation

In [ ]:
# #Removing punctuation AND Casing
#( Casing: Est-ce vraiment utile ? Est-ce qu'on ne va pas perdre les Acronymes de vue ?)
new_tokenized = []
for token in tokenized:
    new_tokenized.append([ word.lower() for word in token if word.isalpha()])

In [ ]:
print(new_tokenized[4])

['le', 'premier', 'alinéa', 'de', 'l', 'article', 'du', 'code', 'de', 'la', 'sécurité', 'sociale', 'est', 'complété', 'par', 'une', 'phrase', 'ainsi', 'rédigée', 'elles', 'sont', 'universelles', 'pendant', 'plus', 'de', 'cinquante', 'ans', 'notre', 'politique', 'familiale', 'a', 'reposé', 'sur', 'le', 'principe', 'de', 'l', 'universalité', 'cela', 'signifie', 'qu', 'elle', 's', 'adressait', 'à', 'tous', 'les', 'français', 'sans', 'distinction', 'sociale', 'elle', 'reposait', 'sur', 'l', 'idée', 'que', 'chaque', 'enfant', 'à', 'naître', 'est', 'une', 'chance', 'et', 'une', 'richesse', 'pour', 'la', 'france', 'pour', 'son', 'avenir', 'quel', 'que', 'soient', 'les', 'ressources', 'dont', 'disposent', 'les', 'mettre', 'en', 'place', 'ce', 'principe', 'd', 'universalité', 'la', 'politique', 'familiale', 'appelle', 'des', 'outils', 'dits', 'de', 'redistribution', 'horizontale', 'c', 'des', 'mécanismes', 'de', 'solidarité', 'des', 'familles', 'sans', 'enfant', 'envers', 'les', 'familles', 'av

In [ ]:
#Stop words

#On importe les mots "inutiles" (stopwords) du langage français
from nltk.corpus import stopwords 
stop_words = stopwords.words("french")
print(stop_words[:20])

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur']


In [ ]:
#On importe nos propres stopwords
SW = pd.read_csv('Added_stop_words.csv')
SW = list(SW['StopWords'])
print("NOS MOTS 'INUTILES' : \n", SW)

NOS MOTS 'INUTILES' : 
 ['alinéa', 'amendement', 'º', 'L.', 'loi', 'présent', 'rapport', '1', '2020', 'II', 'phrase', '2', 'Gouvernement', 'I.', '575', 'organismes', '3', '2019', 'secteur', 'non', 'mot', 'mesure', 'Etat', 'Objet', 'objet', 'compte', 'situation', 'ans', 'propose', '4', 'III', 'également', 'congé', 'ainsi', 'afin', 'tel', 'cet']


In [ ]:
#Stop Word removal
print("AVANT : \n", tokenized[4], "\n")
final_SW = SW + stop_words
tokenized = []
for token in new_tokenized:
    tokenized.append([ word for word in token if word not in final_SW])
print("APRÈS : \n", tokenized[4], "\n")

print("MOT ENLEVÉS : \n", final_SW) # On affiche ci-dessous tous les mots que l'on a retirés

AVANT : 
 ['Le', 'premier', 'alinéa', 'de', 'l', '’', 'article', 'L.', '521‑1', 'du', 'code', 'de', 'la', 'sécurité', 'sociale', 'est', 'complété', 'par', 'une', 'phrase', 'ainsi', 'rédigée', ':', '«', 'Elles', 'sont', 'universelles', '.', '»', 'Pendant', 'plus', 'de', 'cinquante', 'ans', ',', 'notre', 'politique', 'familiale', 'a', 'reposé', 'sur', 'le', 'principe', 'de', 'l', '’', 'universalité', '.', 'Cela', 'signifie', 'qu', '’', 'elle', 's', '’', 'adressait', 'à', 'tous', 'les', 'Français', ',', 'sans', 'distinction', 'sociale', '.', 'Elle', 'reposait', 'sur', 'l', '’', 'idée', 'que', 'chaque', 'enfant', 'à', 'naître', 'est', 'une', 'chance', 'et', 'une', 'richesse', 'pour', 'la', 'France', ',', 'pour', 'son', 'avenir', ',', 'quel', 'que', 'soient', 'les', 'ressources', 'dont', 'disposent', 'les', 'parents.Pour', 'mettre', 'en', 'place', 'ce', 'principe', 'd', '’', 'universalité', ',', 'la', 'politique', 'familiale', 'appelle', 'des', 'outils', 'dits', 'de', 'redistribution', 'hor

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ben/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
#Lemmatization: on reduit les mots à leur racine
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

amdt_clean= []
for token in tokenized:
    amdt_clean.append([lemmatizer.lemmatize(word) for word in token])
print(amdt_clean[4])

['premier', 'article', 'code', 'sécurité', 'sociale', 'complété', 'rédigée', 'elles', 'universelles', 'pendant', 'plus', 'cinquante', 'politique', 'familiale', 'a', 'reposé', 'principe', 'universalité', 'cela', 'signifie', 'adressait', 'tous', 'français', 'sans', 'distinction', 'sociale', 'reposait', 'idée', 'chaque', 'enfant', 'naître', 'chance', 'richesse', 'france', 'avenir', 'quel', 'ressources', 'dont', 'disposent', 'mettre', 'place', 'principe', 'universalité', 'politique', 'familiale', 'appelle', 'outils', 'dit', 'redistribution', 'horizontale', 'mécanismes', 'solidarité', 'familles', 'sans', 'enfant', 'envers', 'familles', 'enfants', 'quel', 'revenus', 'parent', 'naissance', 'enfant', 'effet', 'porter', 'atteinte', 'niveau', 'principe', 'universalité', 'a', 'mi', 'mal', 'sou', 'précédent', 'quinquennat', 'notamment', 'travers', 'modulation', 'allocation', 'familiales', 'bai', 'successives', 'quotient', 'familial', 'faisant', 'politique', 'familiale', 'politique', 'sociale', 'co

In [ ]:
df = pd.DataFrame(amdt_clean)
#print(df.head(10))
df.to_csv('amdt_clean.csv')  #Chaque ligne est un amendement

# Les mots les plus cités

In [ ]:
#Counting
from collections import Counter

most = []
for token in amdt_clean:
    bow = Counter(token)
    most.append(bow.most_common(30))


df = pd.DataFrame(most)
print(df.head(10))
df.to_csv('most_common_word_per_amdt.csv')  #Chaque ligne est un amendement 


                     0                 1                2   \
0         (salariés, 7)  (entreprises, 4)       (moins, 4)   
1       (naissance, 15)      (enfant, 14)       (prime, 9)   
2  (expérimentation, 5)      (article, 4)        (code, 3)   
3          (service, 7)      (article, 5)        (code, 4)   
4        (politique, 5)      (sociale, 4)   (familiale, 4)   
5       (allocation, 6)   (familiales, 5)  (modulation, 5)   
6       (allocation, 4)     (adoptées, 3)   (naissance, 3)   
7       (allocation, 4)     (adoptées, 3)   (naissance, 3)   
8           (charge, 9)      (patient, 8)        (pied, 6)   
9     (substitution, 5)   (pharmacien, 4)  (médicament, 4)   

                   3                    4                  5               6   \
0          (prime, 3)  (exceptionnelle, 3)       (article, 3)       (mise, 3)   
1        (sociale, 6)       (politique, 6)       (article, 5)       (date, 5)   
2        (service, 3)           (cette, 2)     (notamment, 2)     (effets,

# TF IDF
Term frequency - inverse document frequency

In [ ]:
#Mise sous forme 'corpus'
corpus = []
for amdt1 in amdt_clean:
    temp = ' '.join(amdt1)
    corpus.append(temp)
    temp = ''
    
#Vectorization - Term Frequency in Global Corpus
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
tf = vectorizer.fit_transform(corpus)

print(corpus[2:4])

['compléter suivante gouvernement remet parlement échéance cette période expérimentation évaluation portant notamment effets contemporanéité crédit impôt mentionné article sexdecies code général impôts participation financière bénéficiaires prestations mentionnées article code action sociale familles part coûts induits application i article prestataires définis article code travail participant expérimentation autre part louable expérimenter dispositifs permettant plus avoir décalage entre moment où dépense réalisée celui où aide perçues personnes recourant service domicile notamment plus vulnérables entre elles nécessaire mesurer éventuels effets induits négatifs telles mesures effet certains saad facturent prix participation financière légale prévue apa pch convient mesurer effet cette expérimentation prix service conséquence reste ailleurs expérimentation prévoit contemporanéité apa pch organisé centre cesu or ore déjà conseils départementaux peuvent verser directement mécanisme tier

In [ ]:
#Features
feature_names = vectorizer.get_feature_names()
print(feature_names[:10])

['aa', 'aah', 'aasctel', 'ab', 'abaissant', 'abaisse', 'abaissement', 'abaisser', 'abaissés', 'abandon']


In [ ]:
#Génération du corpus index
import re
corpus_index = amdt['uid'].tolist()
print(corpus_index[:5])
#amdt['uid'].nunique() 

['AMANR5L15PO420120B2296P0D1N000001', 'AMANR5L15PO420120B2296P0D1N000002', 'AMANR5L15PO420120B2296P0D1N000005', 'AMANR5L15PO420120B2296P0D1N000008', 'AMANR5L15PO420120B2296P0D1N000010']


In [ ]:
## LAST STOP HERE: POURQUOI A-T-ON UNIQUEMENT DES NAN ?

#IDF - Inverse Document Frequencies
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize and fit TfidfVectorizer
vectorizer = TfidfVectorizer(norm=None)
tf_idf_scores = vectorizer.fit_transform(corpus)
#print(tf_idf_scores)
# get vocabulary of terms
feature_names = vectorizer.get_feature_names()
#print(type(tf_idf_scores.todense()))
print (pd.DataFrame(tf_idf_scores.todense()) )  #ICI on a bien les bonnes valeurs
print(pd.DataFrame(tf_idf_scores.todense()[0][0]))
#print(feature_names)
# create pandas DataFrame with tf-idf scores
df_tf_idf = pd.DataFrame(data = pd.DataFrame(tf_idf_scores.todense()), index=corpus_index, columns=feature_names)
print(df_tf_idf)
#df_tf_idf.fillna(0)
df_tf_idf.replace(np.nan, 0)
#df_tf_idf.reset_index() 
#df_tf_idf.rename_axis('ID') 


      0         1      2      3      4      5      6      7      8      9      \
0       0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1       0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2       0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3       0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4       0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...     ...       ...    ...    ...    ...    ...    ...    ...    ...    ...   
4792    0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4793    0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4794    0.0  5.669292    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4795    0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4796    0.0  0.000000    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

      ...  13052    13053  

,aa,aah,aasctel,ab,abaissant,abaisse,abaissement,abaisser,abaissés,abandon,...,événements,être,êtrepar,île,œnologie,œufs,œuvrant,œuvre,œuvrent,œuvres
AMANR5L15PO420120B2296P0D1N000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMANR5L15PO420120B2296P0D1N000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMANR5L15PO420120B2296P0D1N000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMANR5L15PO420120B2296P0D1N000008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMANR5L15PO420120B2296P0D1N000010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMANR5L15PO717460BTA0353P0D1N000118,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMANR5L15PO717460BTA0353P0D1N000119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMANR5L15PO717460BTA0353P0D1N000120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AMANR5L15PO717460BTA0353P0D1N000121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for n in range(5):#range(len(df_tf_idf)):
    #print(n)
    #print(id)
    main = pd.DataFrame()
    main['Most_frequent_words'] = feature_names
    main['Word_occurrence'] = df_tf_idf.iloc[n]  # MAIN ISSUE HERE: Series definition not taking float but convert2 nan
    print(max(df_tf_idf.iloc[n]))
    print(len(df_tf_idf.iloc[n]))
    print(type(df_tf_idf.iloc[n]))
    print(max(main['Word_occurrence']))
    print(type(main['Word_occurrence']))
    print(type(main))
    print(main.head(5))
    main.sort_values(by =['Word_occurrence'], inplace = True, ascending = False, na_position='last')
    print(main.head(5))

nan
13062
<class 'pandas.core.series.Series'>
nan
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
  Most_frequent_words  Word_occurrence
0                  aa              NaN
1                 aah              NaN
2             aasctel              NaN
3                  ab              NaN
4           abaissant              NaN
  Most_frequent_words  Word_occurrence
0                  aa              NaN
1                 aah              NaN
2             aasctel              NaN
3                  ab              NaN
4           abaissant              NaN
nan
13062
<class 'pandas.core.series.Series'>
nan
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
  Most_frequent_words  Word_occurrence
0                  aa              NaN
1                 aah              NaN
2             aasctel              NaN
3                  ab              NaN
4           abaissant              NaN
  Most_frequent_words  Word_occurrence
0          

In [ ]:
#exclude
#Organizing most common words !Il faut transposer
# tuple_mots = ()
# for n in range(len(df_tf_idf)):
#     print(n)
#     id = df_tf_idf.index[n]
#     print(id)
#     #Creating a DF per amendment
#     main = pd.DataFrame()
#     main['Mots principaux'] = df_tf_idf.columns.T
#     main['Occurence'] = df_tf_idf.iloc[n]
    
    
#     #main['mots'] = df_tf_idf['ID']
    
#     print(max(df_tf_idf.index[n]))
#     #print(main.head(5))
#     #main.sort_values(main[id])

#     #main = main.iloc[:25] #On ne garde que les 25 mots les plus utilisés par amendement
#     #tuple_mots.append(main)

In [ ]:
df_tf_idf.shape

(4797, 13062)

In [ ]:
id = 0

In [ ]:
df_tf_idf.index[0]

'AMANR5L15PO420120B2296P0D1N000001'

In [ ]:
test = df_tf_idf.index[id]